In [ ]:
# Desenvolvimento de ACs para a simulação da propagação de notícias

# Importar as bibliotecas utilizadas no decorrer do programa:
from tomato.classes import cell
from matplotlib import pyplot as plt
import numpy as np

# Determinação das regras de transição:
class NewsPropagation(cell.CellTemplate):
    
    def update(self, state_matrix):
        
        self.state_matrix = state_matrix
        
        # p_1: probabilidade que a informação seja propagada
        p_1 = np.random.normal(0, 0.65)
        
        # R: threshold
        R = 1.05
        
        # Apenas as células que NÃO aceitaram a informação mudam (uma vez com a informação, elas permanecem com ela)
        if self.value == 0:
            
            if self.information_accepted < 3:
                    p_1 = p_1*1.5
            
            # Para que a célula adquira informação, m*p_1 >= R
            if (self.information_accepted*p_1) >= R:
                    self.value = 1
                    
            else:
                self.value = 0
    
    @property
    def neighbors(self):
        return self.moore_neighborhood
        # Considera-se a vizinhança de MOORE

    @property
    def information_accepted(self):
        return self.neighbors.count(1)
        # Conta o n° de vizinhos que aceitaram a informação

    @property
    def information_not_accepted(self):
        return self.neighbors.count(0)
        # Conta o n° de vizinhos que NÃO aceitaram a informação

    # Definição das cores de cada tipo de célula (accepted/not accepted)
    @staticmethod
    def display(value):
        if value == 1:
            return (0, 0, 0)
        # Células que aceitaram a informação: PRETAS
        
        elif value == 0:
            return (255, 255, 255)
        # Células que não aceitaram a informação: BRANCAS

    @staticmethod
    def from_display(value):
        if (value == (0, 0, 0)).all():
            return 1
        
        elif (value == (255, 255, 255)).all():
            return 0

In [ ]:
import tomato as tt
from tomato.functions import utils

rule = NewsPropagation

CELL_SIZE = 40
dimensions = (100, 100)

state_matrix = np.random.choice(a=[0, 1], size = dimensions, p=[0.50, 0.50])
# state_matrix = np.zeros(dimensions)
# state_matrix[49, 49] = 1

initial_state_matrix = state_matrix

board = tt.Board(rule, cell_size=CELL_SIZE)
board.load_state(initial_state_matrix)

# Criando vetores numpy para armazenar as populações de cada espécie de célula
news_pop = np.zeros(100)
no_news_pop = np.zeros(100)

while board.generation < 100:
    # Matriz com os estados de cada célula:
    state_matrix = board.state_matrix

    news_pop[board.generation] = np.count_nonzero(state_matrix == 1)
    no_news_pop[board.generation] = np.count_nonzero(state_matrix == 0)

    # Iterar a simulação depois de terminada a nossa coleta de dados da geração
    board.update()

# Avisando que a simulação acabou e fazendo umas estatísticas básicas
print("Prontinho!")
print(f"news_pop: initial {news_pop[0]} | final {news_pop[-1]} | avg {np.mean(news_pop)}")
print(f"no_news_pop: initial {no_news_pop[0]} | final {no_news_pop[-1]} | avg {np.mean(no_news_pop)}")

initial_news = news_pop[0]/(no_news_pop[0] + news_pop[0])
initial_no_news = no_news_pop[0]/(no_news_pop[0] + news_pop[0])

final_news = news_pop[-1]/(no_news_pop[-1] + news_pop[-1])
final_no_news = no_news_pop[-1]/(no_news_pop[-1] + news_pop[-1])

In [ ]:
fig, ax = plt.subplots()
ax.plot(news_pop, color="red", label="News")
ax.plot(no_news_pop, color="blue", label="No news")

ax.set_title("Evolução das populações")
ax.set_xlabel("Geração")
ax.set_ylabel("População")
ax.legend()

In [ ]:
mycolors = ["blue", "red"]
mylabels = ["News", "No news"]
myexplode = [0.05, 0.05]

# Populações iniciais:
y = np.array([initial_news, initial_no_news])
plt.title("Populações iniciais")
plt.pie(y, labels = mylabels, startangle = 90, explode = myexplode, shadow = True, autopct='%1.1f%%', colors = mycolors)
plt.show()

# Populações finais:
y = np.array([final_news, final_no_news])
plt.title("Populações finais")
plt.pie(y, labels = mylabels, startangle = 90, explode = myexplode, shadow = True, autopct='%1.1f%%', colors = mycolors)
plt.show() 

In [ ]:
import tomato as tt
from tomato.functions import utils

rule = NewsPropagation

CELL_SIZE = 1
dimensions = (100, 100)

state_matrix = np.random.choice(a=[0, 1], size = dimensions, p=[0.50, 0.50])
# state_matrix = np.zeros(dimensions)
# state_matrix[49, 49] = 1

initial_state_matrix = state_matrix

board = tt.Board(rule, cell_size=CELL_SIZE)
board.load_state(initial_state_matrix)

board.start(
    state_matrix, 
    inline = True, 
    generations = 100, 
    generate_figures = True, 
    generate_figures_dir = "Trabalho_Final", # Especificar o nome da pasta
    generate_gif = True # Criar um gif com as imagens ao final da execução
)

board.save_png("Modelo_2_GIF.png")